# **3) Symbol typing**

In [1]:
from pathlib import Path
import os, sys, platform, json, subprocess

ROOT = Path.cwd().resolve().parents[0]

if (ROOT / "src").exists():
    sys.path.insert(0, str(ROOT))

print("Project root:", ROOT)
print("Python:", sys.version)
print("OS:", platform.platform())

Project root: D:\IIT BBS\Job Resources\fiolabs\diagram-intel
Python: 3.11.13 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:03:15) [MSC v.1929 64 bit (AMD64)]
OS: Windows-10-10.0.26100-SP0


In [3]:
from src.config.loader import load_cfg
cfg = load_cfg()
cfg.symbols

{'use_vlm_on_meso': True, 'max_tiles_vlm': 60, 'min_labels_in_tile': 1, 'select_top_by_labels': 200, 'max_new_tokens': 180, 'temperature': 0.0}

In [3]:
# Classifier
from src.vision.runners.symbol_classifier import classify_meso_tiles
classify_meso_tiles(cfg)

d:\Anaconda\envs\diagram-intel\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
Loading checkpoint shards: 100%|██

2025-08-27 06:26:12.463 | INFO     | src.vision.runners.symbol_classifier:classify_meso_tiles:175 - [symbols] wrote 60 candidates → D:\IIT BBS\Job Resources\fiolabs\diagram-intel\data\processed\components\candidates.index.json


#### **Inspecting index**

In [4]:
from src.utils.io import read_json
from pathlib import Path
idx = read_json(Path(cfg.paths.processed)/"components"/"candidates.index.json")
len(idx), idx[:5]

(60,
 [{'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
   'page': 1,
   'id': 'wiring diagram_Main Distribution Panel_Fio Labs:1:meso:r001c005',
   'tile': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\interim\\tiles\\meso\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\tile_r001_c005.png',
   'type': None,
   'conf': 0.0},
  {'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
   'page': 1,
   'id': 'wiring diagram_Main Distribution Panel_Fio Labs:1:meso:r001c006',
   'tile': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\interim\\tiles\\meso\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\tile_r001_c006.png',
   'type': None,
   'conf': 0.0},
  {'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
   'page': 1,
   'id': 'wiring diagram_Main Distribution Panel_Fio Labs:1:meso:r001c000',
   'tile': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\interim\\tiles\\meso\\wiring diagram_Main Distribution Panel_Fio

#### **Peek candidate**

In [5]:
from src.utils.io import read_json
from pathlib import Path
cand_dir = Path(cfg.paths.processed)/"components"/"candidates"
one = next(cand_dir.rglob("*.json"))
one, read_json(one)

(WindowsPath('D:/IIT BBS/Job Resources/fiolabs/diagram-intel/data/processed/components/candidates/wiring diagram_Main Distribution Panel_Fio Labs/page-1/meso_r000_c001.json'),
 {'id': 'wiring diagram_Main Distribution Panel_Fio Labs:1:meso:r000c001',
  'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
  'page': 1,
  'tile_path': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\interim\\tiles\\meso\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\tile_r000_c001.png',
  'tile_bbox': [870.0, 0.0, 1894.0, 1024.0],
  'type': None,
  'confidence': 0.0,
  'ports_expected': [],
  'notes': 'parse_error',
  'alternatives': [],
  'labels_context': ['To', 'From', 'Inverter', 'TB1-4', 'MCB11 32A'],
  'source_model': 'qwen2_vl_2b'})

#### **Merge pass that clusters overlapping candidates**

In [4]:
from src.post.merge_candidates import run_merge
run_merge(cfg)

from src.utils.io import read_json
from pathlib import Path
idx = read_json(Path(cfg.paths.processed)/"components"/"merged"/"merged.index.json")
len(idx), idx[:5]

2025-08-27 11:23:51.461 | INFO     | src.post.merge_candidates:run_merge:182 - [merge] wiring diagram_Main Distribution Panel_Fio Labs page-1: 60 → 4 merged components
2025-08-27 11:23:51.464 | INFO     | src.post.merge_candidates:run_merge:185 - [merge] wrote 4 entries → D:\IIT BBS\Job Resources\fiolabs\diagram-intel\data\processed\components\merged\merged.index.json


(4,
 [{'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
   'page': 1,
   'path': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\processed\\components\\merged\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\comp_610625.json',
   'type': None,
   'conf': 0.0,
   'n_sources': 17},
  {'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
   'page': 1,
   'path': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\processed\\components\\merged\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\comp_438942.json',
   'type': None,
   'conf': 0.0,
   'n_sources': 28},
  {'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
   'page': 1,
   'path': 'D:\\IIT BBS\\Job Resources\\fiolabs\\diagram-intel\\data\\processed\\components\\merged\\wiring diagram_Main Distribution Panel_Fio Labs\\page-1\\comp_712463.json',
   'type': None,
   'conf': 0.0,
   'n_sources': 14},
  {'pdf': 'wiring diagram_Main Distribution Panel_Fio Labs',
   'page': 1,
   